In [3]:
import pyecharts
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.datasets import register_url
import pandas as pd
from datetime import date
import datetime
from dateutil import tz
import json
import os
import json
from pyecharts.globals import ChartType


# global variables, the value is current file path
BASE_PATH=os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
update_date = datetime.datetime.now(tz=tz.gettz(
    'Asia/Shanghai')).strftime('%Y-%m-%d %H:%M:%S')

In [5]:
def fill_province(df):
    df["Province/State"] = df["Province/State"].fillna(df["Country/Region"])

In [6]:
remove_list = ["README.md", ".gitignore"]
path = BASE_PATH + "/../COVID-19/csse_covid_19_data/csse_covid_19_time_series"

#Read Global Confirmed CSV data
global_confirmed_df = pd.read_csv(path + "/time_series_covid19_confirmed_global.csv")
fill_province(global_confirmed_df)

# Filter out China Confirmed CSV data
china_confirmed_df = global_confirmed_df[global_confirmed_df["Country/Region"] == "China"]


In [5]:
#Create Global Daily Increase Dataframe
global_add_df = global_confirmed_df.loc[:,:'1/22/20'] #Take all rows, and columns up until '1/22/20', first day of record

last_column = '1/22/20'
for column in global_confirmed_df.loc[:,'1/23/20':]:
    global_add_df[column] = global_confirmed_df[column] - global_confirmed_df[last_column]
    last_column = column

#Filter out China Daily Increase CSV Data
china_add_df = global_add_df[global_add_df['Country/Region']=='China']

#Observe
global_add_df.head()
china_add_df.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20
56,Anhui,China,31.8257,117.2264,1,8,6,24,21,10,...,0,0,0,0,0,0,0,0,0,0
57,Beijing,China,40.1824,116.4142,14,8,14,5,27,12,...,0,0,0,0,0,0,0,0,0,0
58,Chongqing,China,30.0572,107.8740,6,3,18,30,18,35,...,0,0,0,0,0,0,0,0,0,0
59,Fujian,China,26.0789,117.9874,1,4,5,8,17,24,...,1,0,0,0,0,1,1,3,0,0
60,Gansu,China,35.7518,104.2861,0,2,0,2,3,7,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Read Global Death CSV data
global_deaths_df = pd.read_csv(path + "/time_series_covid19_deaths_global.csv")
fill_province(global_deaths_df)

# Filter out China Death CSV data
china_deaths_df = global_deaths_df[global_deaths_df["Country/Region"] == "China"]

# Observe
china_deaths_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20
56,Anhui,China,31.8257,117.2264,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
57,Beijing,China,40.1824,116.4142,0,0,0,0,0,1,...,9,9,9,9,9,9,9,9,9,9
58,Chongqing,China,30.0572,107.8740,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
59,Fujian,China,26.0789,117.9874,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
60,Gansu,China,35.7518,104.2861,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [9]:
# Read Global Recover CSV data
global_recovered_df = pd.read_csv(path + "/time_series_covid19_recovered_global.csv")
fill_province(global_recovered_df)

# Filter out China Recover CSV data
china_recovered_df = global_recovered_df[global_recovered_df["Country/Region"] == "China"]

# Observe
china_recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20
43,Anhui,China,31.8257,117.2264,0,0,0,0,0,0,...,985,985,985,985,985,985,985,985,985,985
44,Beijing,China,40.1824,116.4142,0,0,1,2,2,2,...,923,924,924,924,924,924,924,924,924,926
45,Chongqing,China,30.0572,107.8740,0,0,0,0,0,0,...,577,577,577,577,577,577,577,577,577,577
46,Fujian,China,26.0789,117.9874,0,0,0,0,0,0,...,365,365,365,365,365,365,365,365,365,365
47,Gansu,China,35.7518,104.2861,0,0,0,0,0,0,...,167,167,167,167,167,167,167,167,167,167


In [7]:
def save_location_json(df, file_name):
    # Loop through global confirmed df, Save Long&Lat information for each Province/State
    # Loop through US confirmed df, Save Long&Lat information for each City 
    data_dict = {}
    if 'global' in file_name:
        for _ , row in df.iterrows():   
            data_dict[row['Province/State']] = [row['Long'], row['Lat']]
    elif 'US' in file_name:
        for _ , row in df.iterrows(): 
            data_dict[row['Combined_Key']] = [row['Long'], row['Lat']]
    
    # Save data to local as json file
    with open(file_name + ".json", "w")as f:
        f.write(json.dumps(data_dict))

# Process Global confirmed df 
save_location_json(global_confirmed_df, "global_location")


In [1]:
# Generate Geo Map
def generate_geomap(title, maptype, data, pieces, day, sum):
    """
    :param title: 图的左上角标题
    :param data:  填充到图片上的数据，数据为一个列表，列表里面为每个坐标的元组
                  例子：[("广州", 300), ...("武汉", 600)]
    :param pieces: 参考pyeacharts的opts.VisualMapOpts配置的pieces参数
    :param day:   右下角显示的日期
    :param sum:   右下角显示的累计确诊数
    """
    _map = (
    Geo()
        # 定义地图类型为中国地图，并初始化
        .add_schema(maptype=maptype, itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="#111"))
        # 从json文件中读取每个地点，和这个地点的经纬度数据
        .add_coordinate_json(json_file="global_location.json")
        # 定义标题，数据，地图上的标记图类型
        .add(title, data, type_=ChartType.EFFECT_SCATTER)
        # 不显示每个地点数值
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            # 定义气泡图大小，及如何按数值分割等
            visualmap_opts=opts.VisualMapOpts(type_='size', range_opacity=2, is_piecewise=True, pieces=pieces),
            title_opts=opts.TitleOpts(title="新型冠状病毒全国疫情地图",
                                  subtitle="更新时间：{}".format(update_date)),
            legend_opts=opts.LegendOpts(is_show=False),
            # 定义右下角贴图
            graphic_opts=[opts.GraphicGroup(graphic_item=opts.GraphicItem(
                        rotation=JsCode("Math.PI / 4"),
                        bounding="raw",
                        right=200,
                        bottom=110,
                        z=100),
                        children=[
                        opts.GraphicRect(
                            graphic_item=opts.GraphicItem(
                                left="center", top="center", z=100
                            ),
                            graphic_shape_opts=opts.GraphicShapeOpts(
                                width=400, height=50
                            ),
                            graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                fill="rgba(0,0,0,0.3)"
                            ),
                        ),
                        opts.GraphicText(
                            graphic_item=opts.GraphicItem(
                                left="center", top="center", z=100
                            ),
                            graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                text=day + "\n" + title + ": " + sum,
                                font="bold 26px Microsoft YaHei",
                                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                    fill="#fff"
                                ),
                            ),
                        ),
                    ],
                    )
                    ],
            
        )
    
    )
    return _map

def generate_timeline(timeline, maptype, df, title, pieces):
    # 获取筛选出来的中国确诊数据的所有列名称
    columns = [column for column in df]
    # 观察数据可得，从第四列开始便是每日的确诊数据，每天为一列
    for date in columns[4:]:
        # 列名的日期不符合阅读习惯，重新组合日期
        day_split = date.split("/")
        day = "20" + day_split[-1] + "-" + day_split[0].zfill(2) + "-" + day_split[1].zfill(2)
        confirm = []
        # 摘取需要的字段， date为当前循环的日期字段
        day_confirmed_df = df[["Province/State", date]]
        # 循环摘取出来的DataFrame，返回结果为元组
        for row in day_confirmed_df.itertuples():
            # 元组第一个元素为索引，第二个为Province/State字段，第三个为当前循环date字段
            confirm.append((row[1], row[2]))
        # 获取当前循环date字段数据汇总值
        sum = day_confirmed_df[date].sum()
        # 生成Geo图
        _map = generate_geomap(title, maptype, confirm, pieces, day, str(sum))
        # 将图添加到当前循环日期的时间序列中，时间标签为重新组合的日期
        timeline.add(_map, day)

In [ ]:
# 定义图片上方标签对象
tab = Tab()
# 定义时间轴对象,初始化时间轴大小
tl_confirmed = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_confirmed.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
tl_add = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_add.add_schema(axis_type='time', is_timeline_show=True, play_interval=1500, 
              label_opts=opts.LabelOpts(is_show=False))

# 定义时间轴对象,初始化时间轴大小
tl_recovered = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_recovered.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
# 定义时间轴对象,初始化时间轴大小
tl_deaths = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_deaths.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
# 定义Geo图pieces参数
pieces=[
        {"min": 1, "max": 100, "symbolSize": 5, "color": "#FFA07A"}, 
        {"min": 101, "max": 500, "symbolSize": 10, "color": "rgb(255,99,71)"},
        {"min": 501, "max": 1000, "symbolSize": 15, "color": "rgb(255,99,71)"},
        {"min": 1001, "max": 1500, "symbolSize": 20, "color": "#CD5C5C"},
        {"min": 1501, "max": 2000, "symbolSize": 25, "color": "red"},
        {"min": 2001, "symbolSize": 50, "color": "#8B0000"},
        ]


generate_timeline(tl_confirmed, "china", china_confirmed_df, "确诊", pieces)
tab.add(tl_confirmed, '累计确诊')

generate_timeline(tl_add, "china", china_add_df, "新增", pieces)
tab.add(tl_add, '当日新增')

# 将治愈气泡改成绿色
pieces_cured=[
        {"min": 1, "max": 100, "symbolSize": 5, "color": "#7FFFD4"}, 
        {"min": 101, "max": 500, "symbolSize": 10, "color": "#C1FFC1"},
        {"min": 501, "max": 1000, "symbolSize": 15, "color": "#B4EEB4"},
        {"min": 1001, "max": 1500, "symbolSize": 20, "color": "#4EEE94"},
        {"min": 1501, "max": 2000, "symbolSize": 25, "color": "#00FF7F"},
        {"min": 2001, "symbolSize": 50, "color": "green"},
        ]
generate_timeline(tl_recovered, "china", china_recovered_df, "治愈", pieces_cured)
tab.add(tl_recovered, "治愈")

generate_timeline(tl_deaths, "china", china_deaths_df, "死亡", pieces)
tab.add(tl_deaths, '死亡')

tab.render_notebook()

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

...

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts

全球时间序列

In [25]:
# 定义图片上方标签对象
tab = Tab()
# 定义时间轴对象,初始化时间轴大小
tl_confirmed = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_confirmed.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
tl_add = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_add.add_schema(axis_type='time', is_timeline_show=True, play_interval=1500, 
              label_opts=opts.LabelOpts(is_show=False))

# 定义时间轴对象,初始化时间轴大小
tl_recovered = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_recovered.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
# 定义时间轴对象,初始化时间轴大小
tl_deaths = Timeline(
    init_opts=opts.InitOpts(
        width='1000px',
        height='700px'))
tl_deaths.add_schema(axis_type='time', is_timeline_show=True, play_interval=100, 
              label_opts=opts.LabelOpts(is_show=False))
# 定义Geo图pieces参数
pieces=[
        {"min": 1, "max": 10000, "symbolSize": 5, "color": "#FFA07A"}, 
        {"min": 10001, "max": 50000, "symbolSize": 10, "color": "rgb(255,99,71)"},
        {"min": 50001, "max": 200000, "symbolSize": 15, "color": "rgb(255,99,71)"},
        {"min": 200001, "max": 500000, "symbolSize": 20, "color": "#CD5C5C"},
        {"min": 500001, "max": 1000000, "symbolSize": 25, "color": "red"},
        {"min": 1000001, "symbolSize": 50, "color": "#8B0000"},
        ]


generate_timeline(tl_confirmed, "world", global_confirmed_df, "确诊", pieces)
tab.add(tl_confirmed, '累计确诊')

generate_timeline(tl_add, "world", global_add_df, "新增", pieces)
tab.add(tl_add, '当日新增')

# 将治愈气泡改成绿色
pieces_cured=[
        {"min": 1, "max": 100, "symbolSize": 5, "color": "#7FFFD4"}, 
        {"min": 101, "max": 1000, "symbolSize": 10, "color": "#C1FFC1"},
        {"min": 1001, "max": 10000, "symbolSize": 15, "color": "#B4EEB4"},
        {"min": 10001, "max": 50000, "symbolSize": 20, "color": "#4EEE94"},
        {"min": 50001, "max": 100000, "symbolSize": 25, "color": "#00FF7F"},
        {"min": 100001, "symbolSize": 50, "color": "green"},
        ]
generate_timeline(tl_recovered, "world", global_recovered_df, "治愈", pieces_cured)
tab.add(tl_recovered, "治愈")

pieces_deaths=[
        {"min": 1, "max": 100, "symbolSize": 5, "color": "#FFA07A"}, 
        {"min": 101, "max": 1000, "symbolSize": 10, "color": "rgb(255,99,71)"},
        {"min": 1001, "max": 5001, "symbolSize": 15, "color": "rgb(255,99,71)"},
        {"min": 5001, "max": 20000, "symbolSize": 20, "color": "#CD5C5C"},
        {"min": 20001, "max": 100000, "symbolSize": 25, "color": "red"},
        {"min": 100001, "symbolSize": 50, "color": "#8B0000"},
        ]

generate_timeline(tl_deaths, "world", global_deaths_df, "死亡", pieces_deaths)
tab.add(tl_deaths, '死亡')

tab.render_notebook()

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\composite_charts\timeline.py:12: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 C

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaco

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaco

C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Users\sijia\anaco

NameError: name 'global_add_df' is not defined